In [20]:
import os
import glob

pref='metrics/120338-BC23'
dir_list = glob.glob(pref+'*')
dir_list.sort()
dir_list

['metrics/120338-BC23.alignment_summary.txt',
 'metrics/120338-BC23.alignment_summary_metrics',
 'metrics/120338-BC23.duplicate_metrics',
 'metrics/120338-BC23.gc_bias.detail_metrics',
 'metrics/120338-BC23.gc_bias.summary_metrics',
 'metrics/120338-BC23.insert_size_metrics',
 'metrics/120338-BC23.quality_distribution_metrics',
 'metrics/120338-BC23.quality_yield_metrics.txt',
 'metrics/120338-BC23.raw_wgs_metrics',
 'metrics/120338-BC23.unmapped.quality_yield_metrics',
 'metrics/120338-BC23.wgs_metrics']

In [108]:
import pandas as pd
import h5py

def readMetricsFile(file):
    func=None
    metricsClass=None
    cvg=[]
    with open(file) as f:
        for line in f:
            if line.startswith('## htsjdk.samtools.metrics.StringHeader'):
                header=next(f).strip().split(' ')
                func=header[1]
                #print("FUNC:",func)
                prms=[(h.split('=')[0],h.split('=')[1]) for h in header if len(h.split('='))==2]
                #print("PRMS:",len(prms))
                next(f)
            if line.startswith('## METRICS CLASS'):
                metricsClass=line.strip().split('\t')[1].split('.')[-1]
                #print("CLASS:",metricsClass)
                cat=next(f).strip().split('\t')
                val=next(f).strip().split('\t')
                metrics=[(c,v) for (c,v) in zip(cat,val)]
                #print("METRICS:",len(metrics))
            if line.startswith('## HISTOGRAM') and metricsClass and metricsClass.endswith('WgsMetrics'):
                cat=next(f).strip().split('\t')
                cvg=[[],[]]
                while len(cvg[0])<=200:
                    row=next(f).strip().split('\t')
                    if len(row)<2: break
                    cvg[0].append(int(row[0]))
                    cvg[1].append(int(row[1]))

    if func==None or metricsClass==None:
        return None,[],None,[],None
    return func,prms,metricsClass,metrics,cvg

prmsDf=pd.DataFrame()
metricsDf=pd.DataFrame()
h5outfile=pref+'.h5'

i=0
j=0
print('PREF:',pref)
for file in dir_list:   
    fn=file[len(pref)+1:]
    if fn.endswith('tsv') or fn.endswith('h5'):
        continue
    if fn.endswith('.txt'):
        fn=fn[:-4]
    print('FILE:',fn)
    func,prms,metricsClass,metrics,cvg=readMetricsFile(file)
    #print()
    
    l=len(prms)
    block=pd.DataFrame({'File':[fn]*l,
                        'Function':[func]*l,
                        'Parameter':[p[0] for p in prms],
                        'Value':[p[1] for p in prms],
                       },index=range(i,i+l)) 
    i=i+l
    prmsDf=pd.concat([prmsDf,block])
    
    l=len(metrics)
    block=pd.DataFrame({'File':[fn]*l,
                        'Class':[metricsClass]*l,
                        'Parameter':[m[0] for m in metrics],
                        'Value':[m[1] for m in metrics],
                       },index=range(j,j+l)) 
    j=j+l
    metricsDf=pd.concat([metricsDf,block])
    
    if cvg:
        cvgDf=pd.DataFrame({'Count':cvg[1]},index=cvg[0])
        cvgDf.to_hdf(h5outfile, key=fn+"_cvg")
    

prmsDf.to_csv(pref+'.params.tsv',sep='\t')
prmsDf.to_hdf(h5outfile, key="params")
metricsDf.to_csv(pref+'.metrics.tsv',sep='\t')
metricsDf.to_hdf(h5outfile, key="metrics")
print('OUT:',h5outfile)


PREF: metrics/120338-BC23
FILE: alignment_summary
FILE: alignment_summary_metrics
FILE: duplicate_metrics
FILE: gc_bias.detail_metrics
FILE: gc_bias.summary_metrics
FILE: insert_size_metrics
FILE: quality_distribution_metrics
FILE: quality_yield_metrics
FILE: raw_wgs_metrics
FILE: unmapped.quality_yield_metrics
FILE: wgs_metrics
OUT: metrics/120338-BC23.h5


In [106]:
cvgDf

,Count
0,103163350
1,8018404
2,6174729
3,5539036
4,5343639
...,...
196,580
197,600
198,617
199,587


In [81]:
metricsDf

,File,Class,Parameter,Value
0,alignment_summary,AlignmentSummaryMetrics,CATEGORY,UNPAIRED
1,alignment_summary,AlignmentSummaryMetrics,TOTAL_READS,690081010
2,alignment_summary,AlignmentSummaryMetrics,PF_READS,690081010
3,alignment_summary,AlignmentSummaryMetrics,PCT_PF_READS,1
4,alignment_summary,AlignmentSummaryMetrics,PF_NOISE_READS,0
...,...,...,...,...
153,wgs_metrics,CollectWgsMetrics$WgsMetrics,PCT_80X,0.000325
154,wgs_metrics,CollectWgsMetrics$WgsMetrics,PCT_90X,0.000219
155,wgs_metrics,CollectWgsMetrics$WgsMetrics,PCT_100X,0.000146
156,wgs_metrics,CollectWgsMetrics$WgsMetrics,HET_SNP_SENSITIVITY,0.959369
